**Описание данных**: Японский международный университет опросил своих студентов. Исследование показало, что иностранные студенты имеют более высокий риск проблем с психическим здоровьем (тест PHQ-9), и что социальные связи (тест SCS) и аккультурационный стресс (стресс, связанный с присоединением к новой культуре, тест ASISS) являются предикторами депрессии.

**План решение**:
1. Смотрим на данные
2. Считаем агрегатные функции для теста PHQ-9, SCS, ASISS по типам студентов
3. Считаем корреляции между временем пребывания, SCS, ASISS и PHQ-9

**Инструменты**: WHERE, GROUP BY, ОКОННЫЕ ФУНКЦИИ, ROLLUP, СТАТИСТИЧЕСКИЕ ФУНКЦИИ  

| Field Name    | Description                                      |
| ------------- | ------------------------------------------------ |
| `inter_dom`     | Типы студентов (иностранные или местные)   |
| `japanese_cate` | Знание японского                     |
| `english_cate`  | Знание английского                    |
| `academic`      | Бакалавриат или магистратура |
| `age`           | Возраст                           |
| `stay`          | Пребывание в годах                 |
| `todep`         | Общий балл депрессии (тест PHQ-9)           |
| `tosc`          | Общий балл социальных связей (тест SCS)   |
| `toas`          | Общий балл аккультурационного стресса (тест ASISS) |

In [ ]:
SELECT inter_dom, japanese_cate, english_cate, academic, age, stay, todep, tosc, toas
FROM students

,inter_dom,japanese_cate,english_cate,academic,age,stay,todep,tosc,toas
0,Inter,Average,High,Grad,24.0,5.0,0.0,34.0,91.0
1,Inter,High,High,Grad,28.0,1.0,2.0,48.0,39.0
2,Inter,High,High,Grad,25.0,6.0,2.0,41.0,51.0
3,Inter,Low,Average,Grad,29.0,1.0,3.0,37.0,75.0
4,Inter,Low,Average,Grad,28.0,1.0,3.0,37.0,82.0
...,...,...,...,...,...,...,...,...,...
281,None,None,None,None,NaN,NaN,NaN,NaN,NaN
282,None,None,None,None,NaN,NaN,NaN,NaN,NaN
283,None,None,None,None,NaN,NaN,NaN,NaN,NaN
284,None,None,None,None,NaN,NaN,NaN,NaN,NaN


In [ ]:
SELECT inter_dom, COUNT(*) as count_inter_dom, SUM(count_inter_dom) OVER() as total_records
FROM students
GROUP BY inter_dom

,inter_dom,count_inter_dom,total_records
0,Inter,201,286.0
1,Dom,67,286.0
2,None,18,286.0


In [ ]:
SELECT inter_dom, COUNT(*) as count_inter_dom, SUM(count_inter_dom) OVER() as total_records
FROM students
WHERE inter_dom IS NOT NULL
GROUP BY inter_dom

,inter_dom,count_inter_dom,total_records
0,Inter,201,268.0
1,Dom,67,268.0


In [ ]:
SELECT COALESCE(inter_dom, 'All') AS inter_dom, MIN(todep) AS min_phq, MAX(todep) AS max_phq, ROUND(AVG(todep), 2) AS avg_phq
FROM students
WHERE inter_dom IS NOT NULL
GROUP BY ROLLUP (inter_dom)

,inter_dom,min_phq,max_phq,avg_phq
0,All,0.0,25.0,8.19
1,Inter,0.0,25.0,8.04
2,Dom,0.0,23.0,8.61


In [ ]:
SELECT COALESCE(inter_dom, 'All') AS inter_dom, MIN(tosc) AS min_scs, MAX(tosc) AS max_scs, ROUND(AVG(tosc), 2) AS avg_scs
FROM students
WHERE inter_dom IS NOT NULL
GROUP BY ROLLUP (inter_dom)

,inter_dom,min_scs,max_scs,avg_scs
0,All,8.0,48.0,37.47
1,Inter,11.0,48.0,37.42
2,Dom,8.0,48.0,37.64


In [ ]:
SELECT COALESCE(inter_dom, 'All') AS inter_dom, MIN(toas) AS min_as, MAX(toas) AS max_as, ROUND(AVG(toas), 2) AS avg_as
FROM students
WHERE inter_dom IS NOT NULL
GROUP BY ROLLUP (inter_dom)

,inter_dom,min_as,max_as,avg_as
0,All,36.0,145.0,72.38
1,Inter,36.0,145.0,75.56
2,Dom,36.0,112.0,62.84


In [ ]:
SELECT stay,
       ROUND(AVG(todep), 2) AS average_phq,
       ROUND(AVG(tosc), 2) AS average_scs,
       ROUND(AVG(toas), 2) AS average_as
FROM students
WHERE inter_dom = 'Inter'
GROUP BY stay
ORDER BY stay DESC

,stay,average_phq,average_scs,average_as
0,10.0,13.00,32.00,50.00
1,8.0,10.00,44.00,65.00
2,7.0,4.00,48.00,45.00
3,6.0,6.00,38.00,58.67
4,5.0,0.00,34.00,91.00
5,4.0,8.57,33.93,87.71
6,3.0,9.09,37.13,78.00
7,2.0,8.28,37.08,77.67
8,1.0,7.48,38.11,72.80


In [ ]:
SELECT ROUND(CORR(stay, average_phq),2) as stay_phq, ROUND(CORR(average_phq, average_scs), 2) as phq_scs, ROUND(CORR(average_phq, average_as), 2) as phq_as
FROM df # из CTE

,stay_phq,phq_scs,phq_as
0,0.23,-0.19,-0.29
